# Dicoding Belajar Machine Learning Untuk Pemula Final Project : Image Classification

- **Nama**         : Azarya Yehezkiel Pinondang Sipahutar
- **Email**         : azaryaemc@gmail.com
- **ID Dicoding**   : azarya_yehezkiel

In [1]:
# Import the necessary modules
import zipfile,os
import shutil
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from sklearn.model_selection import train_test_split


import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image


2.15.0


## Data Preparation

In this section, we load the images from our dataset and perform some preprocessing such as rescaling the images. We also split our dataset into training and validation sets.

In [61]:
# Define the path of the zip file
local_zip = 'rockpaperscissors.zip'

# Open the zip file in read mode
zip_ref = zipfile.ZipFile(local_zip, 'r')

# Extract all the contents of the zip file in /tmp directory
zip_ref.extractall('/tmp')

# Close the ZipFile object
# zip_ref.close()

# Define the base directory where the images are located
base_dir = '/tmp/rockpaperscissors/rps-cv-images'

# Define the directory for the training data
rock_dir = os.path.join(base_dir, 'rock')
paper_dir = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors') 

### Split Train(60%) Test (40%) Data

In [62]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Split the data by train/test for each directory
train_rock_dir, val_rock_dir = train_test_split(os.listdir(rock_dir), test_size = 0.4)
train_paper_dir, val_paper_dir = train_test_split(os.listdir(paper_dir), test_size = 0.4)
train_scissors_dir, val_scissors_dir = train_test_split(os.listdir(scissors_dir), test_size = 0.4)

In [63]:
# Function to move files
def move_files(files, src_dir, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)  # Ensure the directory exists
    for file in files:
        shutil.move(os.path.join(src_dir, file), os.path.join(dst_dir, file))

# Move the files
move_files(train_rock_dir, rock_dir, os.path.join(train_dir, 'rock'))
move_files(val_rock_dir, rock_dir, os.path.join(validation_dir, 'rock'))

move_files(train_paper_dir, paper_dir, os.path.join(train_dir, 'paper'))
move_files(val_paper_dir, paper_dir, os.path.join(validation_dir, 'paper'))

move_files(train_scissors_dir, scissors_dir, os.path.join(train_dir, 'scissors'))
move_files(val_scissors_dir, scissors_dir, os.path.join(validation_dir, 'scissors'))

In [ ]:
os.listdir(train_dir)

In [ ]:
os.listdir(validation_dir)

In [117]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',
)

train_generator = train_datagen.flow_from_directory(
    train_dir,  # should contain 3 subdirectories, one for each class
    target_size=(200, 200),
    batch_size=25,
    class_mode='categorical'  # 'categorical' for multi-class labels
)

Found 1814 images belonging to 3 classes.


In [118]:
test_datagen = ImageDataGenerator(
                    rescale=1./255
)


validation_generator = test_datagen.flow_from_directory(
    validation_dir,  # should contain 3 subdirectories, one for each class
    target_size=(200, 200),
    batch_size=25,
    class_mode='categorical'  # 'categorical' for multi-class labels
)

Found 1378 images belonging to 3 classes.


In [86]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for classes present in the training and validation datasets

0 : paper
1 : rock
2 : scissors


In [87]:
labels

{0: 'paper', 1: 'rock', 2: 'scissors'}

## Model Building

Here we build our image classification model. We're using a Convolutional Neural Network (CNN) which is commonly used in image classification tasks.

In [88]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),


    tf.keras.layers.Dense(3, activation='softmax')
])

In [105]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_18 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 17, 17, 128)      

In [108]:
model.compile(optimizer=tf.optimizers.RMSprop(),
              loss='kullback_leibler_divergence',
              metrics=['accuracy'])

## Model Training

Now that our model is built, we can train it using our training data. We also validate our model using the validation data.

In [109]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=2
)

Epoch 1/15
91/91 - 18s - loss: 0.0560 - accuracy: 0.9851 - val_loss: 0.0135 - val_accuracy: 0.9971 - 18s/epoch - 201ms/step
Epoch 2/15
91/91 - 17s - loss: 0.0410 - accuracy: 0.9890 - val_loss: 0.0105 - val_accuracy: 0.9971 - 17s/epoch - 189ms/step
Epoch 3/15
91/91 - 17s - loss: 0.0404 - accuracy: 0.9923 - val_loss: 0.0073 - val_accuracy: 0.9978 - 17s/epoch - 185ms/step
Epoch 4/15
91/91 - 16s - loss: 0.0337 - accuracy: 0.9884 - val_loss: 0.0051 - val_accuracy: 0.9985 - 16s/epoch - 180ms/step
Epoch 5/15
91/91 - 16s - loss: 0.0233 - accuracy: 0.9917 - val_loss: 0.0183 - val_accuracy: 0.9949 - 16s/epoch - 180ms/step
Epoch 6/15
91/91 - 16s - loss: 0.0352 - accuracy: 0.9906 - val_loss: 0.0035 - val_accuracy: 0.9978 - 16s/epoch - 178ms/step
Epoch 7/15
91/91 - 16s - loss: 0.0224 - accuracy: 0.9934 - val_loss: 0.2106 - val_accuracy: 0.9673 - 16s/epoch - 180ms/step
Epoch 8/15
91/91 - 16s - loss: 0.0224 - accuracy: 0.9912 - val_loss: 0.0037 - val_accuracy: 0.9985 - 16s/epoch - 179ms/step
Epoch 9/